# Increasing the Maximum Message Size for gRPC


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Helm](https://github.com/kubernetes/helm)
 - [Minikube](https://github.com/kubernetes/minikube) version v0.24.0 or greater
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)


# Create Cluster

Start minikube and ensure custom resource validation is activated and there is 5G of memory. 

**2018-06-13** : At present we find the most stable version of minikube across platforms is 0.25.2 as there are issues with 0.26 and 0.27 on some systems. We also find the default VirtualBox driver can be problematic on some systems to we suggest using the [KVM2 driver](https://github.com/kubernetes/minikube/blob/master/docs/drivers.md#kvm2-driver).

Your start command would then look like:
```
minikube start --vm-driver kvm2 --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC
```

# Setup

In [1]:
!kubectl create namespace seldon

namespace "seldon" created


In [2]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding "kube-system-cluster-admin" created


# Install Helm

In [3]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount "tiller" created
clusterrolebinding "tiller" created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


## Start seldon-core

Install the custom resource definition

In [5]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

NAME:   seldon-core-crd
LAST DEPLOYED: Sat Sep  8 10:02:25 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     0s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          0s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s


NOTES:
NOTES: TODO




In [6]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon \
    --set ambassador.enabled=true

NAME:   seldon-core
LAST DEPLOYED: Sat Sep  8 10:04:42 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ClusterRoleBinding
NAME    AGE
seldon  0s

==> v1/Service
NAME                          TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)                        AGE
seldon-core-ambassador        NodePort   10.110.189.27   <none>       8080:32124/TCP                 0s
seldon-core-ambassador-admin  NodePort   10.103.225.230  <none>       8877:32484/TCP                 0s
seldon-core-seldon-apiserver  NodePort   10.99.72.60     <none>       8080:32479/TCP,5000:31269/TCP  0s
seldon-core-redis             ClusterIP  10.101.46.175   <none>       6379/TCP                       0s

==> v1beta1/Deployment
NAME                                DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-core-ambassador              1        1        1           0          0s
seldon-core-seldon-apiserver        1        1        1           0          0s
seldon-core-seldon-cluster-manager  1     

Check all services are running before proceeding.

In [7]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador 

deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-seldon-apiserver" successfully rolled out
Waiting for rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward to API Gateway**

REST:

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

GRPC:

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8004:5000
```

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8005:8080
```

Install gRPC modules for the prediction protos.

In [8]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

In [10]:
!pygmentize resources/model.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-model"
    },
    "spec": {
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                                }
                            }
                        ],
                     

## Create Seldon Deployment

Deploy the runtime graph to kubernetes.

In [6]:
!kubectl apply -f resources/model.json -n seldon

seldondeployment "seldon-model" created


Get the status of the SeldonDeployment. **When ready the replicasAvailable should be 1**.

In [15]:
!kubectl get seldondeployments seldon-model -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:test-deployment-example-svc-orch replicas:1 replicasAvailable:1] map[name:test-deployment-example-classifier-0 replicas:1 replicasAvailable:1]]]

## Get predictions - no grpx max message size

In [9]:
from seldon_utils import *
API_GATEWAY_REST="localhost:8003"
API_GATEWAY_GRPC="localhost:8004"
API_AMBASSADOR="localhost:8005"

Send a small request which should suceed.

In [10]:
grpc_request_api_gateway('oauth-key','oauth-secret',API_GATEWAY_REST,API_GATEWAY_GRPC,10)

{"access_token":"522e6dd3-2232-44a7-a547-e3babc939ef0","token_type":"bearer","expires_in":42264,"scope":"read write"}
meta {
  puid: "fj2rftu976hrmde87v0dcmkhv7"
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.0787334522874084
  }
}



Test via Ambassador

In [11]:
grpc_request_ambassador("seldon-model",API_AMBASSADOR,data_size=10,rows=1)

meta {
  puid: "qacuk4o0qk7dvvo963t8o1svo1"
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.0898729704574218
  }
}



Send a large request which will be above the default gRPC message size and will fail.

In [17]:
grpc_request_api_gateway('oauth-key','oauth-secret',API_GATEWAY_REST,API_GATEWAY_GRPC,1000000)

{"access_token":"50094bf3-3141-48bf-958a-69a57c1896bd","token_type":"bearer","expires_in":43181,"scope":"read write"}


_Rendezvous: <_Rendezvous of RPC that terminated with (StatusCode.CANCELLED, Received RST_STREAM with error code 8)>

In [12]:
grpc_request_ambassador("seldon-model",API_AMBASSADOR,data_size=1000000,rows=1)

_Rendezvous: <_Rendezvous of RPC that terminated with (StatusCode.UNAVAILABLE, upstream connect error or disconnect/reset before headers)>

In [14]:
!kubectl delete -f resources/model.json

seldondeployment "seldon-model" deleted


In [13]:
!helm delete seldon-core --purge

release "seldon-core" deleted


# Allowing larger gRPC messages

Recreate seldon-core with extra annotation for the API Gateway

In [15]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon \
    --set ambassador.enabled=true \
    -f grpc_size_values.yaml

NAME:   seldon-core
LAST DEPLOYED: Sat Sep  8 10:31:47 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/RoleBinding
NAME    AGE
seldon  0s

==> v1/Service
NAME                          TYPE       CLUSTER-IP     EXTERNAL-IP  PORT(S)                        AGE
seldon-core-ambassador        NodePort   10.108.32.98   <none>       8080:32409/TCP                 0s
seldon-core-ambassador-admin  NodePort   10.106.70.248  <none>       8877:32597/TCP                 0s
seldon-core-seldon-apiserver  NodePort   10.104.51.168  <none>       8080:32546/TCP,5000:30907/TCP  0s
seldon-core-redis             ClusterIP  10.98.60.20    <none>       6379/TCP                       0s

==> v1/ClusterRoleBinding
NAME    AGE
seldon  0s

==> v1beta1/Role
NAME          AGE
ambassador    0s
seldon-local  0s

==> v1beta1/RoleBinding
NAME        AGE
ambassador  0s

==> v1beta1/Deployment
NAME                                DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-core-ambassador              1 

Wait for seldon core deployment to be running

In [16]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador 

deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-seldon-apiserver" successfully rolled out
Waiting for rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-core-ambassador" successfully rolled out


Now we change our SeldonDeployment to include a annotation for max grpx message size.

In [17]:
!pygmentize resources/model_grpc_size.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-model"
    },
    "spec": {
        "annotations": {
	    "seldon.io/grpc-max-message-size":"10000000"
        },
        "name": "test-deployment",
        "oauth_key": "oauth-key",
        "oauth_secret": "oauth-secret",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier_grpc:1.0",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
                     

In [18]:
!kubectl create -f resources/model_grpc_size.json -n seldon

seldondeployment "seldon-model" created


**Make sure you rerun the port forwards above as a new seldon core API gateway and Ambassador will have started**

In [27]:
!kubectl get seldondeployments seldon-model -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[name:test-deployment-grpc-size-svc-orch replicas:1 replicasAvailable:1] map[name:test-deployment-grpc-size-classifier-0 replicas:1 replicasAvailable:1]]]

Send a large message. This time it should succeed.

In [29]:
grpc_request_api_gateway('oauth-key','oauth-secret',API_GATEWAY_REST,API_GATEWAY_GRPC,data_size=1000000,rows=1)

{"access_token":"a4929490-a3c9-4d1a-956d-9c173a22ba8a","token_type":"bearer","expires_in":43199,"scope":"read write"}
meta {
  puid: "uv6fgur0ggvjntduisrgdccvdn"
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08190040853130452
  }
}



Send a request via ambassador. This should also succeed.

In [30]:
grpc_request_ambassador("seldon-model",API_AMBASSADOR,data_size=10,rows=1)

meta {
  puid: "a8lgbq67prn8huq4mmdjgur5js"
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08562220593812069
  }
}

